In [2]:
!pip install --upgrade pip setuptools



In [3]:
!pip install -q cassio datasets langchain openai tiktoken

In [5]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 85.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.0 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.33
    Uninstalling langchain-core-0.3.33:
      Successfully uninstalled langchain-core-0.3.33
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.5
    Uninstalling langchain-text-splitters-0.3.5:
      Successfully uninstalled langchain-text-splitters-0.3.5
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.17
    Uninstalling langchain-0.3.17:
      Successfully uninstalled langchain-0.3.17


Import the packages

In [6]:
# LangChain components to use
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings

# Support for dataset retrieval with Hugging Face
from datasets import load_dataset

# With CassIO, the engine powering the Astra DB integration in LangChain,
# you will also initialize the DB connection:
import cassio


In [7]:
!pip install PyPDF2

In [8]:
from PyPDF2 import PdfReader

Setup

Provide your secrets

In [ ]:
ASTRA_DB_APPLICATION_TOKEN = "AstraCS:KLQmJDgJfEQhTHesdRftkYGJAfm:69b674fee5aa4fe9c423205caa063e3a275e8285b969938bacbdff766a305f09"
ASTRA_DB_ID = "bh34dfe89-ad04-4127-bb22-c6885416ea03"

OPENAI_API_KEY = "sk-proj-YKMitMOmtVfghjkiu3XC4jYCxs7ySHmt6qfEru3azDP1azYhesdfghjn8cUIDnIvus_-T1rfSaIiJvTEXL6eoT3BlbkFJoLiuzxG2Tfg3WoMk41hIRTIQ21jIiTRl4OtjKfKmxBhdQzQFcciiZ-B0hGz-2V40DAwq-ICakA"

In [11]:
pdfreader = PdfReader('/content/Bsdsf21a028_Lab1_Report.pdf')

In [12]:
from typing_extensions import Concatenate

# Read text from PDF
raw_text = ''
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        raw_text += content


In [13]:
raw_text

'Facebook\nData\nPerformance\nAnalysis\nReport\nIntroduction\nThis\nreport\naims\nto\nanalyze\nthe\nperformance\nof\nSQL\nqueries\non\nvarious\ndatasets\nsimulating\nFacebook-related\ndata\nsuch\nas\nuser\nprofiles,\nposts,\ncomments,\nand\ninteractions.\nThe\ngoal\nis\nto\nevaluate\nthe\nperformance\nof\neach\nquery\nacross\ndifferent\ndata\nsizes,\nusing\nexecution\ntimes\nas\na\nmetric\nfor\ncomparison.\nTable\nDescriptions\nand\nData\nPopulation\n:\nObjective\n:\nCreate\nand\npopulate\ntables\nto\nstore\nFacebook-related\ndata,\nreflecting\nrealistic\nscenarios.\nTables\nCreated\nand\nPurposes\nof\nTables:\n1.\nUserProfiles\n:\nStores\nuser\nprofile\ninformation,\nsuch\nas\nuser_id\n,\nname\n,\nemail\n,\ndate_of_birth\n,\nand\njoin_date\n.\n2.\nPosts\n:\nStores\nposts\ncreated\nby\nusers,\nincluding\nfields\nfor\npost_id\n,\nuser_id\n(foreign\nkey),\ncontent\n,\npost_date\n,\nand\nlikes_count\n.\n3.\nComments\n:\nStores\ncomments\nmade\non\nposts,\nwith\nfields\nlike\ncomment_id\n,

In [14]:
cassio.init(token=ASTRA_DB_APPLICATION_TOKEN, database_id=ASTRA_DB_ID)


In [15]:
llm = OpenAI(openai_api_key=OPENAI_API_KEY)
embedding = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)


<ipython-input-15-9c33d76eff6b>:1: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(openai_api_key=OPENAI_API_KEY)
<ipython-input-15-9c33d76eff6b>:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)


Create LangChain vector store...backed by Astra DB!

In [16]:
astra_vector_store = Cassandra(
    embedding=embedding,
    table_name="qa_mini_demo",
    session=None,
    keyspace=None,
)


In [17]:
from langchain.text_splitter import CharacterTextSplitter

# We need to split the text using Character Text Split such that it should be manageable
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 800,
    chunk_overlap = 200,
    length_function = len,
)

texts = text_splitter.split_text(raw_text)


In [20]:
texts[:80]

['Facebook\nData\nPerformance\nAnalysis\nReport\nIntroduction\nThis\nreport\naims\nto\nanalyze\nthe\nperformance\nof\nSQL\nqueries\non\nvarious\ndatasets\nsimulating\nFacebook-related\ndata\nsuch\nas\nuser\nprofiles,\nposts,\ncomments,\nand\ninteractions.\nThe\ngoal\nis\nto\nevaluate\nthe\nperformance\nof\neach\nquery\nacross\ndifferent\ndata\nsizes,\nusing\nexecution\ntimes\nas\na\nmetric\nfor\ncomparison.\nTable\nDescriptions\nand\nData\nPopulation\n:\nObjective\n:\nCreate\nand\npopulate\ntables\nto\nstore\nFacebook-related\ndata,\nreflecting\nrealistic\nscenarios.\nTables\nCreated\nand\nPurposes\nof\nTables:\n1.\nUserProfiles\n:\nStores\nuser\nprofile\ninformation,\nsuch\nas\nuser_id\n,\nname\n,\nemail\n,\ndate_of_birth\n,\nand\njoin_date\n.\n2.\nPosts\n:\nStores\nposts\ncreated\nby\nusers,\nincluding\nfields\nfor\npost_id\n,\nuser_id\n(foreign\nkey),\ncontent\n,\npost_date\n,\nand\nlikes_count\n.\n3.',
 'as\nuser_id\n,\nname\n,\nemail\n,\ndate_of_birth\n,\nand\njoin_date\n.\n2.\nPo

Load the dataset into the vector store

In [21]:
astra_vector_store.add_texts(texts[:80])

print("Inserted %i headlines." % len(texts[:80]))

astra_vector_index = VectorStoreIndexWrapper(vectorstore=astra_vector_store)


Inserted 8 headlines.


Run the QA cycle

In [22]:
first_question = True

while True:
    if first_question:
        query_text = input("\nEnter your question (or type 'quit' to exit): ").strip()
    else:
        query_text = input("\nWhat's your next question (or type 'quit' to exit): ").strip()

    if query_text.lower() == "quit":
        break

    if query_text == "":
        continue

    first_question = False

    print("\nQUESTION: \"%s\"" % query_text)
    answer = astra_vector_index.query(query_text, llm=llm).strip()
    print("ANSWER: \"%s\"\n" % answer)

    print("FIRST DOCUMENTS BY RELEVANCE:")
    for doc, score in astra_vector_store.similarity_search_with_score(query_text, k=4):
        print("  [%0.4f] \"%s ...\"" % (score, doc.page_content[:84]))



Enter your question (or type 'quit' to exit): what is userId

QUESTION: "what is userId"


ANSWER: "userId is a foreign key used to identify and link users to their respective profiles, posts, comments, and interactions in the database. It is used as a unique identifier for each user."

FIRST DOCUMENTS BY RELEVANCE:
  [0.8493] "various
row
counts:
10,000,
50,000,
and
100,000
for
UserProfiles
,
Posts
,
Comments
 ..."
  [0.8493] "various
row
counts:
10,000,
50,000,
and
100,000
for
UserProfiles
,
Posts
,
Comments
 ..."
  [0.8464] "as
user_id
,
name
,
email
,
date_of_birth
,
and
join_date
.
2.
Posts
:
Stores
posts
 ..."
  [0.8464] "as
user_id
,
name
,
email
,
date_of_birth
,
and
join_date
.
2.
Posts
:
Stores
posts
 ..."

What's your next question (or type 'quit' to exit): what are the total  likes on posts

QUESTION: "what are the total  likes on posts"


ANSWER: "I don't have enough information to answer that question as the total likes on posts would depend on the specific posts being referenced in the query."

FIRST DOCUMENTS BY RELEVANCE:


  [0.8929] "various
row
counts:
10,000,
50,000,
and
100,000
for
UserProfiles
,
Posts
,
Comments
 ..."
  [0.8929] "various
row
counts:
10,000,
50,000,
and
100,000
for
UserProfiles
,
Posts
,
Comments
 ..."
  [0.8869] "posts.
SELECT
post_id,
content,
likes_count
FROM
Posts
ORDER
BY
likes_count
DESC
OFF ..."
  [0.8869] "posts.
SELECT
post_id,
content,
likes_count
FROM
Posts
ORDER
BY
likes_count
DESC
OFF ..."

What's your next question (or type 'quit' to exit): quit
